In the class we try to predict the restaurant rating using the dot product. One thing we found was that it predict a lot of non-sense rating like negative number or 6. In this homework, we are going to fix this problem

The problem with dot product is that it is unbounded. So we need to bound it between 0 and 5. The most common way to do this is to use logistic function to turn $(-\infty, \infty)$ to a bounded region.
$$ \theta(s) = \frac{1}{1 + e^{-s}}$$

1) Given the restaurant attribute $\vec{\rho}^{(r)}$ and person preference $\vec{\pi}^{(p)}$. Write down the prediction formula which gives the output in the range of $(0,5)$.

Hint: use dot product and logistic function then scale it properly.

In [12]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [1]:
def scale(s):
    return (1./(1 + np.exp(-s))) * 5

In [53]:
def guess(R, P):
    return scale(np.dot(R.T, P))

2) Write down the cost function with your prediction formula above.

In [3]:
def cost(R, P):
    return sum((T - guess(R, P))**2)

3) (Optional) Show that if your predition formula is (Do it on paper by hand. It's actually just a chain rule.)
$$Guess_{r,p} = a \theta(\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}) + d$$



Then the derivative is given by
$$
	\frac{\partial{c}}{\partial{\pi^{(p)}_i}} =
	\sum_r 2 h_{rp} \left[ a \cdot \frac{1}{1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)} } }  + d - T_{rp} \right] \cdot \frac{ a e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} }{\left( 1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} \right)^2} \rho^{(r)}_i
$$
and
$$
	\frac{\partial{c}}{\partial{\rho^{(r)}_i}} =
	\sum_p 2 h_{rp} \left[ a \cdot \frac{1}{1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)} } }  + d - T_{rp} \right] \cdot \frac{ a e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} }{\left( 1 + e^{ -\vec{\rho}^{(r)} \cdot \vec{\pi}^{(p)}} \right)^2} \pi^{(p)}_i
$$



4) Write the above two equations in matrix form given that matrix

$$
    S_{rp} = 2 h \otimes \left[ a \cdot \frac{1}{1 + e^{ -R^T P } }  + d - T \right] \cdot \frac{ a e^{ -R^T P} }{\left( 1 + e^{ -R^T P} \right)^2}
$$
where the exponential is element-wise exponential(yes there is such thing as exponential of matrix but that's not what we want).

The partial derivative should look super simple in terms of $S$.

In [ ]:
dcost /dP = R x HT
dcost /dR = P x HT.T

5) Write down the update rule for R and P. Use $a$ and $d$ you found in 1.

In [15]:
def theta(s):
    return (1./(1 + np.exp(-s)))

In [37]:
def find_R_P(R, P):
    l = 0.001
    
    for i in xrange(100000):
        RP = np.dot(R.T, P)
        HT = (2 * h * l) * (  (a * theta(RP)) + d - T  ) * (  (a * np.exp(-RP))/(1 + np.exp(-RP))**2  )
        
        P = P - np.dot(R, HT)
        R = R - np.dot(P, HT.T)
    
    return R, P

6) Given the rating matrix we use in class, use this new prediction function and update rule to find $R$ and $P$.

In [73]:
def read_rating():
    all_ratings = []
    all_defined = []
    
    with open('Exercise 10/rating.csv') as f:
            lines = f.readlines()
            useful_lines = lines[3:]

            names = lines[2].split(',')[2:]
            names = map(lambda x: x.strip(), names)
            rnames = []

            for iline, line in enumerate(useful_lines):
                tokens = line.split(',')
                tokens = map(lambda x: x.strip(), tokens)

                rname = tokens[1]
                ratings = tokens[2:]

                defined = map(lambda x: 0 if (x == '' or x == '"') else 1, ratings)

                def clean_cast(x):
                    return 0 if (x == '' or x == '"') else float(x)

                ratings = map(lambda x: clean_cast(x), ratings)

                all_ratings.append(ratings)
                all_defined.append(defined)

                rnames.append(rname)

            T = np.array(all_ratings)
            H = np.array(all_defined)
            
    return T, H, names, rnames

In [74]:
T, H, names, rnames = read_rating()

7) Use the code we had in exercise to show the prediction table.

In [75]:
npeople = len(names)
nrest = len(rnames)
nfeature = 3
a = 5

In [83]:
np.random.seed(17)
P = np.random.randn(nfeature, npeople)
R = np.random.randn(nfeature, nrest)

In [84]:
def score(R, P):
    return np.sum(H * (T - guess(R, P))**2)

In [85]:
print score(R, P)
np.clip

3018.15353293


<function numpy.core.fromnumeric.clip>

In [86]:
def find_R_P(R, P):
    l = 0.001
    
    for i in xrange(150000):
        RP = np.dot(R.T, P)
        HT = (2 * H * l) * (  (a * theta(RP)) - T  ) * (  (a * np.exp(-RP))/(1 + np.exp(-RP))**2  )
        
        P = P - np.dot(R, HT)
        R = R - np.dot(P, HT.T)
        
        if (i%5000 == 0):
            print i, score(R, P)
    
    return R, P

In [87]:
R, P = find_R_P(R, P)

0 2925.58830903
5000 315.354124917
10000 309.452699042
15000 303.204866854
20000 299.896678182
25000 297.968546358
30000 296.372012793
35000 295.272038933
40000 294.406612697
45000 293.807480187
50000 293.351422204
55000 292.970905577
60000 292.641272778
65000 292.34911536
70000 292.085681636
75000 291.844815103
80000 291.622056613
85000 291.41422143
90000 291.219155093
95000 291.035482088
100000 290.862304368
105000 290.698934275
110000 290.544739094
115000 290.399088835
120000 290.261355902
125000 290.130929117
130000 290.007227516
135000 289.889710022
140000 289.777880326
145000 289.671287797


In [66]:
from IPython.display import HTML

class TableCell:
    
    def __init__(self, text, tc=None, color=None):
        self.text = text
        self.tc = tc
        self.color = color
    
    def to_html(self):
        return '<td>%s</td>'%self.text

#the rating and guess matrix has different convention from the notes so be sure to transpose it first
def maketable(rating, has_rating, guess, restaurants, names):
    n_rests = len(restaurants)
    n_names = len(names)
    tab = np.empty((n_rests+1, n_names+1),dtype='object')
#     guess = scale(guess)
    #print tab.shape

    for irest in range(n_rests):
        tab[irest+1,0] = restaurants[irest]

    for iname in range(n_names):
        tab[0,iname+1] = names[iname]

    for irest in range(n_rests):
        for iname in range(n_names):
            if not has_rating[iname, irest]:
                tab[irest+1, iname+1] = TableCell('<span style="color:red">%3.2f</span>'%(guess[iname, irest]))
            else:
                tab[irest+1, iname+1] = TableCell('<span style="color:blue">%3.2f</span><span style="color:red">(%3.2f)</span>'%(rating[iname, irest], guess[iname, irest]))
    #now convert tab array to nice html table
    nrow, ncol = tab.shape
    t = []
    t.append('<table>')
    for irow in range(nrow):
        t.append('<tr>')
        for icol in range(ncol):
            cell = tab[irow,icol]
            if cell is not None:
                if isinstance(cell,TableCell):
                    t.append(tab[irow, icol].to_html())
                else:
                    t.append('<td>')
                    t.append(tab[irow, icol])
                    t.append('</td>')
            else:
                t.append('<td></td>')
        t.append('</tr>')  
    t.append('</table>')
    return '\n'.join(t)

In [88]:
HTML(maketable(T.T, H.T, guess(R, P).T, rnames, names))

,Piti,Meena,Pitoon,Sohum,Sam,Keng,O,Yok,Kitty,Time,Robroo,Peem,Chanon,Ohm,Sun,Tii,Tow,John,Mint,Opal,Kelly,Jay,Sharik,Ou,PJ,Punawit,Martin,Ploy,Majeed,Bossy,Sea,Pan,Ice,Karn,May,Rin,Peter,Ham,Benz,Billy,Kanat,Sam,Boss,Best
Mai-tok-mai-tak,4.76,3.43,2.00(2.08),3.73,4.83,2.00(3.37),2.00(3.72),3.00(3.41),3.82,3.00(3.22),3.00(3.25),5.00(3.65),2.68,3.00(2.54),2.00(3.03),3.00(3.40),3.00(3.58),2.63,2.00(2.36),3.00(3.06),3.00(3.08),3.00(3.04),3.34,2.00(2.83),0.92,3.95,3.02,4.00(3.04),3.13,3.00(3.03),4.00(3.61),3.28,3.42,3.00(3.11),4.00(3.43),3.52,4.15,3.14,4.00(3.33),4.00(4.05),4.00(3.43),3.44,4.00(3.57),5.00(3.26)
Puttharaksa,4.00(3.69),3.29,4.00(2.62),4.00(3.64),4.00(5.00),3.00(3.42),4.00(3.73),3.00(3.44),3.53,3.00(3.16),3.27,4.00(3.67),3.00(2.88),4.00(3.49),3.00(3.06),4.00(3.55),4.00(3.81),3.00(2.92),3.00(3.43),3.00(3.24),3.00(3.26),3.00(3.07),3.00(3.36),3.00(2.84),0.07,3.65,3.10,3.00(3.04),3.00(2.97),3.01,4.00(3.81),4.00(3.27),4.00(4.11),3.12,4.00(4.06),3.00(3.54),3.50,3.36,1.00(3.33),4.00(3.69),3.00(3.15),4.45,4.00(3.70),4.00(3.19)
Big Mamma,4.97,4.00(3.95),1.77,5.00(4.28),5.00(4.91),3.81,4.38,3.89,4.00(4.41),3.64,3.92,4.26,2.74,0.26,3.34,3.90,2.56,1.22,0.54,3.40,3.22,3.36,3.79,3.03,0.42,4.52,3.34,3.38,3.58,3.35,4.15,3.70,3.94,3.48,4.00(3.93),4.00(4.16),4.72,3.00(3.42),3.82,4.99,3.94,3.53,4.12,3.77
Seefah,1.68,2.00(3.34),4.00(3.18),4.00(3.81),5.00(5.00),3.69,4.00(4.04),3.00(3.71),3.48,5.00(3.28),3.53,4.00(3.97),3.00(3.18),4.05,3.24,4.00(3.93),4.09,3.00(3.09),4.18,3.61,3.59,3.24,4.00(3.60),2.00(2.95),0.00,4.00(3.60),3.34,3.00(3.20),2.94,2.00(3.13),4.24,3.45,4.70,3.30,3.00(4.65),3.82,3.00(2.95),4.00(3.75),3.54,4.00(4.07),3.02,4.91,5.00(4.09),4.00(3.32)
Music Square,2.00(2.22),3.68,3.29,4.21,3.00(5.00),4.00(4.07),4.33,4.00(4.07),4.00(3.88),4.00(3.58),4.00(3.69),4.29,3.40,4.90,4.00(3.52),4.00(4.27),5.00(4.79),4.13,5.00(4.83),3.91,4.00(4.01),3.00(3.51),3.96,2.00(3.13),0.00,5.00(4.02),3.62,3.00(3.45),3.11,3.38,4.00(4.56),3.79,4.86,4.00(3.58),5.00(4.83),4.00(4.11),3.29,4.12,3.87,2.00(1.99),4.00(3.33),4.98,4.00(4.42),3.00(3.58)
Mamma Mia,3.00(2.85),4.00(3.79),3.17,5.00(4.30),5.00(5.00),4.11,4.00(4.47),4.00(4.13),4.04,4.00(3.65),5.00(3.93),5.00(4.41),3.00(3.37),5.00(4.21),3.00(3.56),5.00(4.32),4.00(4.46),3.20,4.00(4.33),4.00(3.95),4.00(3.93),4.00(3.56),4.02,3.00(3.16),0.00,3.00(4.18),4.00(3.66),3.00(3.51),3.26,4.00(3.43),5.00(4.60),3.00(3.85),5.00(4.87),4.00(3.64),5.00(4.84),5.00(4.26),3.69,3.00(4.10),3.00(3.96),5.00(4.60),3.00(3.45),4.97,4.00(4.48),3.71
Srijan,4.00(4.25),3.12,3.00(2.27),3.33,4.46,3.00(3.07),3.41,5.00(3.11),3.40,3.00(2.97),2.00(3.11),4.00(3.33),2.62,2.00(1.44),2.00(2.85),2.00(3.13),2.00(2.65),2.06,1.00(1.73),2.89,2.82,2.86,3.06,3.00(2.71),1.10,4.00(3.50),2.85,2.86,3.00(2.93),2.00(2.84),3.29,4.00(3.01),3.24,2.91,3.00(3.22),3.26,3.68,3.00(2.91),3.00(3.07),4.00(4.47),3.00(3.10),3.14,4.00(3.26),3.00(3.03)
Steak House,0.00,3.00(3.09),5.00,4.00(3.96),4.00(3.99),3.17,5.00,4.81,4.00(3.90),3.31,5.00,5.00,4.35,0.00,4.16,5.00,0.00,1.00(0.00),0.00,4.99,0.00(0.00),4.07,5.00(4.67),3.00(3.16),0.00,3.00(2.96),4.90,4.82,4.91,3.27,5.00,3.00(3.01),5.00,4.32,5.00,5.00,3.00(3.03),2.00(2.15),4.97,5.00,0.17,5.00,5.00,4.99
Anya,1.00(0.65),4.00(3.88),4.00(3.77),4.00(4.45),5.00(5.00),4.00(4.32),4.00(4.73),4.00(4.36),4.00(4.08),4.00(3.79),3.00(4.36),3.00(4.65),4.00(3.67),1.00(1.58),4.00(3.76),4.60,3.59,2.00(1.68),4.00(3.51),4.00(4.28),4.00(4.09),5.00(3.76),5.00(4.24),4.00(3.29),0.00,4.21,4.00(3.93),4.00(3.73),3.00(3.33),4.00(3.58),4.81,4.03,5.00(4.97),3.84,4.97,4.57,3.27,5.00(4.37),4.00(4.20),5.00(5.00),3.00(3.33),5.00,5.00(4.72),4.00(3.94)
Chicky Chic,2.00(2.03),3.00(2.63),2.00(2.06),2.00(2.31),0.00(0.00),2.00(1.70),5.00(4.61),2.00(2.27),3.00(3.31),2.00(2.38),5.00(4.82),4.00(4.01),2.00(1.85),1.00(0.00),2.00(2.28),2.85,0.00,0.00(0.00),2.00(0.00),2.00(2.49),0.00(0.35),2.00(2.28),2.00(2.26),3.00(2.31),3.00(2.97),3.00(3.19),3.00(2.50),2.00(2.73),3.54,3.00(2.32),3.00(2.49),2.06,3.00(3.21),3.00(2.38),3.00(2.93),5.00(4.46),5.00(4.51),2.00(1.09),2.00(2.88